## Tutorial 2: Statistics

While TensorFlow offers some support for statistical inference, TensorFlow-Probability is very strong at this and provides MCMC methods, probability distributions and more.

In [1]:
import zfit
from zfit import z
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## Distributions

There is a whole collection of different distributions to be found in TFP. They have a minimal and well designed interface, which is similar to the SciPy distributions.

In [2]:
tfd = tfp.distributions

In [3]:
cauchy = tfd.Cauchy(loc=1., scale=10.)

In [4]:
sample = cauchy.sample(10)

In [5]:
cauchy.prob(sample)

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.02881326, 0.02045262, 0.00568203, 0.00745806, 0.02504767,
       0.02334153, 0.01715136, 0.02477342, 0.01758285, 0.03176593],
      dtype=float32)>

### Mixtures of PDFs

TensorFlow-Probability also supports creating mixtures of different distributions.

In [6]:
mix = 0.3
mix_gauss_cauchy = tfd.Mixture(
  cat=tfd.Categorical(probs=[mix, 1.-mix]),
  components=[
    cauchy,
    tfd.Normal(loc=+1., scale=0.5),
])

In [7]:
sample_mixed = mix_gauss_cauchy.sample(10)

In [8]:
mix_gauss_cauchy.prob(sample_mixed)

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.4268287 , 0.00808947, 0.40308884, 0.3533951 , 0.50831145,
       0.00361311, 0.21846075, 0.00726999, 0.492639  , 0.00376655],
      dtype=float32)>

### Joint distributions

Furthermore, joint distributions of multiple variables are supported.

In [9]:
joint = tfd.JointDistributionNamed(dict(
    c=             tfd.Cauchy(loc=10., scale=1.),
    n=             tfd.Normal(loc=0, scale=2.),
    m=lambda n, c: tfd.Normal(loc=n, scale=c),
))

In [10]:
sample_joint = joint.sample(10)
sample_joint

{'n': <tf.Tensor: shape=(10,), dtype=float32, numpy=
 array([-0.22471611, -2.605682  ,  0.07820922, -0.31202337, -0.26899937,
        -4.469181  ,  0.5473684 ,  1.0505059 , -2.472432  , -1.3669764 ],
       dtype=float32)>,
 'c': <tf.Tensor: shape=(10,), dtype=float32, numpy=
 array([ 9.745735 , 10.026542 ,  6.842549 , 10.113817 ,  8.2216215,
        10.33216  , 10.035208 ,  8.993729 , 11.222084 ,  8.983006 ],
       dtype=float32)>,
 'm': <tf.Tensor: shape=(10,), dtype=float32, numpy=
 array([-22.037973  , -17.36492   ,   7.328728  ,  -4.3564305 ,
          8.403266  ,  -7.5173616 ,   6.8670096 ,  -5.600155  ,
         16.180342  ,  -0.70785797], dtype=float32)>}

In [11]:
joint.prob(sample_joint)

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.00019817, 0.00036566, 0.00019235, 0.00225489, 0.00042052,
       0.0001741 , 0.00199156, 0.00092746, 0.00010592, 0.00109445],
      dtype=float32)>

### How TFP compares to zfit

TensorFlow-Probability offers a great choice of distributions to build a model. The flexibility in terms of vectorization and parametrization is larger than in zfit. However, they only provide models with analytically known CDFs and lack any numerical normalization or sampling methods. This excludes any more sophisticated model, convolutions and more.

Internally, zfit simply wraps TFP distributions for certain implementations, such as the `Gauss`. There is also a standard wrapper, `WrapDistribution`, that allows to easily wrap any TFP distribution and use it in zfit.